In [1]:
!pip install sqlalchemy psycopg2-binary

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 23.9 MB/s  0:00:00


In [ ]:
import pandas as pd
import os
from sqlalchemy import create_engine, text

from dotenv import load_dotenv
import os

load_dotenv('/Users/Swaminathan/Desktop/all_code_base/eas503_final_project/housing_app_fall25/.env')

# 1. SETUP CONFIGURATION
RENDER_URL = os.getenv("DB_URL")
DATA_PATH = "../data/WA_Fn-UseC_-Telco-Customer-Churn.csv"

def load_telco_data():
    print(f"[1] Loading Telco CSV from {DATA_PATH}...")
    df = pd.read_csv(DATA_PATH)
    
    # Pre-processing: TotalCharges is often read as an object because of empty spaces
    # We convert it to numeric and fill empty values with 0
    df['TotalCharges'] = pd.to_numeric(df['TotalCharges'], errors='coerce').fillna(0)
    
    # Normalize column names to lowercase for Postgres
    df.columns = [c.lower() for c in df.columns]
    return df

def build_3nf_postgres():
    print("=== BUILDING 3NF POSTGRES DATA MODEL (TELCO CHURN) ===")

    # [STEP 1] Load Data
    df = load_telco_data()
    print(f"Loaded {len(df)} rows.")

    # [STEP 2] Create 3NF Splits
    # Table 1: Customers (Demographics)
    df_customers = df[[
        'customerid', 'gender', 'seniorcitizen', 'partner', 'dependents'
    ]].copy()

    # Table 2: Services (Internet and Phone details)
    df_services = df[[
        'customerid', 'phoneservice', 'multiplelines', 'internetservice', 
        'onlinesecurity', 'onlinebackup', 'deviceprotection', 'techsupport', 
        'streamingtv', 'streamingmovies'
    ]].copy()

    # Table 3: Contracts (Financials & Target Variable)
    df_contracts = df[[
        'customerid', 'tenure', 'contract', 'paperlessbilling', 
        'paymentmethod', 'monthlycharges', 'totalcharges', 'churn'
    ]].copy()

    print("Data successfully split into 3 tables (Customers, Services, Contracts).")

    # [STEP 3] Connect to Render Postgres
    engine = create_engine(RENDER_URL)

    # [STEP 4] Create Schema and Upload
    print("\n[STEP 4] Creating schema and inserting data into Render...")
    
    with engine.begin() as conn:
        # Drop existing tables
        conn.execute(text("DROP TABLE IF EXISTS contracts CASCADE;"))
        conn.execute(text("DROP TABLE IF EXISTS services CASCADE;"))
        conn.execute(text("DROP TABLE IF EXISTS customers CASCADE;"))

        # 1. Create Customers Table
        conn.execute(text("""
            CREATE TABLE customers (
                customerid TEXT PRIMARY KEY,
                gender TEXT,
                seniorcitizen INTEGER,
                partner TEXT,
                dependents TEXT
            );
        """))

        # 2. Create Services Table
        conn.execute(text("""
            CREATE TABLE services (
                customerid TEXT PRIMARY KEY,
                phoneservice TEXT,
                multiplelines TEXT,
                internetservice TEXT,
                onlinesecurity TEXT,
                onlinebackup TEXT,
                deviceprotection TEXT,
                techsupport TEXT,
                streamingtv TEXT,
                streamingmovies TEXT,
                CONSTRAINT fk_customer_services FOREIGN KEY(customerid) REFERENCES customers(customerid)
            );
        """))

        # 3. Create Contracts Table
        conn.execute(text("""
            CREATE TABLE contracts (
                customerid TEXT PRIMARY KEY,
                tenure INTEGER,
                contract TEXT,
                paperlessbilling TEXT,
                paymentmethod TEXT,
                monthlycharges REAL,
                totalcharges REAL,
                churn TEXT,
                CONSTRAINT fk_customer_contracts FOREIGN KEY(customerid) REFERENCES customers(customerid)
            );
        """))

    # [STEP 5] Insert Data
    df_customers.to_sql('customers', engine, if_exists='append', index=False)
    df_services.to_sql('services', engine, if_exists='append', index=False)
    df_contracts.to_sql('contracts', engine, if_exists='append', index=False)

    print("\n=== DONE! Render Postgres Database Populated for Telco. ===")

# Run the process
build_3nf_postgres()

# [STEP 6] VERIFY WITH A JOIN
print("\n[STEP 6] Verifying with SQL JOIN...")
engine = create_engine(RENDER_URL)
query = """
SELECT c.gender, s.internetservice, k.tenure, k.monthlycharges, k.churn
FROM customers c
JOIN services s ON c.customerid = s.customerid
JOIN contracts k ON c.customerid = k.customerid
LIMIT 5;
"""
verify_df = pd.read_sql(query, engine)
print(verify_df)

=== BUILDING 3NF POSTGRES DATA MODEL (TELCO CHURN) ===
[1] Loading Telco CSV from ../data/WA_Fn-UseC_-Telco-Customer-Churn.csv...
Loaded 7043 rows.
Data successfully split into 3 tables (Customers, Services, Contracts).

[STEP 4] Creating schema and inserting data into Render...

=== DONE! Render Postgres Database Populated for Telco. ===

[STEP 6] Verifying with SQL JOIN...
   gender internetservice  tenure  monthlycharges churn
0  Female             DSL       9            65.6    No
1    Male             DSL       9            59.9    No
2    Male     Fiber optic       4            73.9   Yes
3    Male     Fiber optic      13            98.0   Yes
4  Female     Fiber optic       3            83.9   Yes
